In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from collections import Counter
from nltk.corpus import stopwords
from stop_words import get_stop_words
from xgboost import XGBClassifier
import joblib
import nltk
from collections import Counter
import tensorflow as tf
import matplotlib.pyplot as plt

stopwords = list(get_stop_words('en'))

def max_repeated_word_count(text):
    words = [word for word in text.split() if word not in stopwords]

    word_counts = Counter(words)
    try:
        return word_counts.most_common(1)[0][1]
    
    except IndexError:
        return 0
        
    return max_count

def get_pos_tags(x):
    tokens = nltk.tokenize.word_tokenize(x)
    tags = nltk.pos_tag(tokens, tagset='universal')
    return Counter(tag for word, tag in tags)

In [2]:
data_0 = pd.read_csv("../input/feedback-train-further/train_further_0.csv")
data_0 = data_0.drop(columns = ['preds5_1', 'preds5_2', 'preds5_3'], axis=1)

data_0_d = pd.read_csv("../input/feedback-train-further-extra-data/train_further_deb_0.csv")
data_0_d = data_0_d.drop(columns = ['essay_id', 'discourse_text', 'discourse_type', 'discourse_effectiveness', 'fold'], axis=1)

data_0_inner = pd.merge(data_0, data_0_d, how='inner', on=['discourse_id'])

# data_0_b = pd.read_csv("../input/feedback-train-further-extra-data/train_further_rob_0.csv")
# data_0_b = data_0_b.drop(columns = ['essay_id', 'discourse_text', 'discourse_type', 'discourse_effectiveness', 'fold'], axis=1)
# data_0_inner = pd.merge(data_0_inner, data_0_b, how='inner', on=['discourse_id'])

print(data_0_inner.shape)

(7353, 21)


In [3]:
data_1 = pd.read_csv("../input/feedback-train-further/train_further_1.csv")
data_1 = data_1.drop(columns = ['preds5_1', 'preds5_2', 'preds5_3'], axis=1)

data_1_d = pd.read_csv("../input/feedback-train-further-extra-data/train_further_deb_1.csv")
data_1_d = data_1_d.drop(columns = ['essay_id', 'discourse_text', 'discourse_type', 'discourse_effectiveness', 'fold'], axis=1)

data_1_inner = pd.merge(data_1, data_1_d, how='inner', on=['discourse_id'])

# data_1_b = pd.read_csv("../input/feedback-train-further-extra-data/train_further_rob_1.csv")
# data_1_b = data_1_b.drop(columns = ['essay_id', 'discourse_text', 'discourse_type', 'discourse_effectiveness', 'fold'], axis=1)
# data_1_inner = pd.merge(data_1_inner, data_1_b, how='inner', on=['discourse_id'])

print(data_1_inner.shape)

(7353, 21)


In [4]:
data_3 = pd.read_csv("../input/feedback-train-further/train_further_3.csv")
data_3 = data_3.drop(columns = ['preds5_1', 'preds5_2', 'preds5_3'], axis=1)

data_3_d = pd.read_csv("../input/feedback-train-further-extra-data/train_further_deb_3.csv")
data_3_d = data_3_d.drop(columns = ['essay_id', 'discourse_text', 'discourse_type', 'discourse_effectiveness', 'fold'], axis=1)

data_3_inner = pd.merge(data_3, data_3_d, how='inner', on=['discourse_id'])

# data_3_b = pd.read_csv("../input/feedback-train-further-extra-data/train_further_rob_3.csv")
# data_3_b = data_3_b.drop(columns = ['essay_id', 'discourse_text', 'discourse_type', 'discourse_effectiveness', 'fold'], axis=1)
# data_3_inner = pd.merge(data_3_inner, data_3_b, how='inner', on=['discourse_id'])

print(data_3_inner.shape)

(7353, 21)


In [5]:
data_4 = pd.read_csv("../input/feedback-train-further/train_further_4.csv")
data_4 = data_4.drop(columns = ['preds5_1', 'preds5_2', 'preds5_3'], axis=1)

data_4_d = pd.read_csv("../input/feedback-train-further-extra-data/train_further_deb_4.csv")
data_4_d = data_4_d.drop(columns = ['essay_id', 'discourse_text', 'discourse_type', 'discourse_effectiveness', 'fold'], axis=1)

data_4_inner = pd.merge(data_4, data_4_d, how='inner', on=['discourse_id'])

# data_4_b = pd.read_csv("../input/feedback-train-further-extra-data/train_further_rob_4.csv")
# data_4_b = data_4_b.drop(columns = ['essay_id', 'discourse_text', 'discourse_type', 'discourse_effectiveness', 'fold'], axis=1)
# data_4_inner = pd.merge(data_4_inner, data_4_b, how='inner', on=['discourse_id'])

print(data_4_inner.shape)

(7353, 21)


In [6]:
data = pd.concat([data_0_inner, data_1_inner, data_3_inner, data_4_inner])
print(data.shape)

(29412, 21)


In [7]:
# # one-time check
# y = data['discourse_effectiveness'].map({'Ineffective': 0, 'Adequate': 1, 'Effective': 2})
# cols_keep = ['preds6_1', 'preds6_2', 'preds6_3']
# X = data[cols_keep]

# print(log_loss(y, X))

In [8]:
# char length
data['char_length'] = data['discourse_text'].apply(lambda x: len(x))
# word count
data['word_count'] = data['discourse_text'].apply(lambda x: len(x.split()))
# avg word length
data['avg_word_length'] = data['char_length'] / data['word_count']
# contains 'source'
data['contains_source'] = data['discourse_text'].apply(lambda x: 'source' in x.lower().split())
# contains 'i'
data['contains_I'] = data['discourse_text'].apply(lambda x: 'i' in x.lower().split())
# repeated word count
data['max_repeated_word_count'] = data['discourse_text'].apply(max_repeated_word_count)

In [9]:
# POS
data["pos_tags"] = data['discourse_text'].apply(lambda x: get_pos_tags(x))
data = pd.concat([data, data["pos_tags"].apply(pd.Series).fillna(0)], axis=1)
data.drop("pos_tags", axis=1, inplace=True)

In [10]:
cols = ['discourse_type', 'char_length', 'word_count', 'avg_word_length', 'contains_source', 'contains_I', 'max_repeated_word_count', 
        'preds2_1', 'preds2_2', 'preds2_3', 
        'preds3_1', 'preds3_2', 'preds3_3', 
        'preds4_1', 'preds4_2', 'preds4_3', 
        'preds5_1', 'preds5_2', 'preds5_3',
        'ADJ', 'ADV', 'PRON', 'NUM', 'CONJ', 
        'discourse_effectiveness']
data = data[cols]

In [11]:
data.head()

,discourse_type,char_length,word_count,avg_word_length,contains_source,contains_I,max_repeated_word_count,preds2_1,preds2_2,preds2_3,...,preds4_3,preds5_1,preds5_2,preds5_3,ADJ,ADV,PRON,NUM,CONJ,discourse_effectiveness
0,Lead,321,55,5.836364,False,False,2,0.470207,0.500918,0.028876,...,0.421165,0.020156,0.424608,0.555236,6.0,6.0,4.0,0.0,0.0,Adequate
1,Position,112,20,5.600000,False,True,1,0.481609,0.510175,0.008216,...,0.478896,0.003002,0.362385,0.634613,2.0,0.0,1.0,0.0,0.0,Adequate
2,Claim,151,28,5.392857,False,True,1,0.353120,0.635564,0.011316,...,0.734519,0.015296,0.369319,0.615385,2.0,2.0,4.0,0.0,0.0,Adequate
3,Claim,39,7,5.571429,False,False,1,0.455436,0.466478,0.078087,...,0.147192,0.087109,0.680018,0.232873,1.0,1.0,2.0,0.0,0.0,Adequate
4,Claim,31,7,4.428571,False,False,1,0.563831,0.419973,0.016196,...,0.603617,0.022393,0.547576,0.430031,0.0,0.0,2.0,0.0,0.0,Adequate


In [12]:
y = data['discourse_effectiveness'].map({'Ineffective': 0, 'Adequate': 1, 'Effective': 2})
X = data.drop(columns = ['discourse_effectiveness'], axis=1)

In [13]:
X_train, X_test, y_train_, y_test_ = train_test_split(X, y, test_size=0.15, random_state=87)

In [14]:
encode_cols = ['discourse_type', 'contains_source', 'contains_I']

for i, col in enumerate(encode_cols):
    labelencoder = LabelEncoder()
    X_train[col] = labelencoder.fit_transform(X_train[col])
    joblib.dump(labelencoder, f'encoder_{i}.pkl')
    X_test[col] = labelencoder.transform(X_test[col])

In [15]:
ss_X = StandardScaler()
X_train = ss_X.fit_transform(X_train)
joblib.dump(ss_X, 'scaler.pkl')
X_test = ss_X.transform(X_test)

y_train = np.zeros((y_train_.shape[0], 3))
for i, v in enumerate(y_train_):
    y_train[i, v] = 1
y_test = np.zeros((y_test_.shape[0], 3))
for i, v in enumerate(y_test_):
    y_test[i, v] = 1

In [16]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax') 
])

2022-08-23 08:15:19.719659: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [17]:
# lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
#     1e-4,
#     decay_steps=10000,
#     decay_rate=0.96,
#     staircase=True)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics = ['categorical_crossentropy'])

In [18]:
class myCallback(tf.keras.callbacks.Callback):
    def __init__(self):
        super().__init__()
        self.best_score = 100
        self.num_model = 0
        
    def on_epoch_end(self, epoch, logs={}):
        score = logs.get('val_categorical_crossentropy')
        if score <= self.best_score:
            self.best_score = score
            self.num_model += 1
            model.save(f'model{self.num_model}')
        if epoch == 19:
            print()
            print(self.best_score)

callbacks = myCallback()

history = model.fit(
            X_train,
            y_train,
            epochs=20,
            validation_data=(X_test, y_test),
            callbacks=[callbacks],
            verbose=2
            )

2022-08-23 08:15:19.910626: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/20
782/782 - 4s - loss: 0.6425 - categorical_crossentropy: 0.6425 - val_loss: 0.5875 - val_categorical_crossentropy: 0.5875


2022-08-23 08:15:24.089673: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


Epoch 2/20
782/782 - 2s - loss: 0.5950 - categorical_crossentropy: 0.5950 - val_loss: 0.5846 - val_categorical_crossentropy: 0.5846
Epoch 3/20
782/782 - 2s - loss: 0.5901 - categorical_crossentropy: 0.5901 - val_loss: 0.5798 - val_categorical_crossentropy: 0.5798
Epoch 4/20
782/782 - 2s - loss: 0.5875 - categorical_crossentropy: 0.5875 - val_loss: 0.5801 - val_categorical_crossentropy: 0.5801
Epoch 5/20
782/782 - 2s - loss: 0.5859 - categorical_crossentropy: 0.5859 - val_loss: 0.5777 - val_categorical_crossentropy: 0.5777
Epoch 6/20
782/782 - 2s - loss: 0.5846 - categorical_crossentropy: 0.5846 - val_loss: 0.5779 - val_categorical_crossentropy: 0.5779
Epoch 7/20
782/782 - 2s - loss: 0.5836 - categorical_crossentropy: 0.5836 - val_loss: 0.5770 - val_categorical_crossentropy: 0.5770
Epoch 8/20
782/782 - 2s - loss: 0.5822 - categorical_crossentropy: 0.5822 - val_loss: 0.5777 - val_categorical_crossentropy: 0.5777
Epoch 9/20
782/782 - 2s - loss: 0.5813 - categorical_crossentropy: 0.5813 - 

In [19]:
# plt.plot(history.history['categorical_crossentropy'])
# plt.plot(history.history['val_categorical_crossentropy'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

In [20]:
model.save('model')